<a href="https://colab.research.google.com/github/bebyakinb/test_assignments_Analyst/blob/master/habr/The_best_medium_difficulty_SQL_interview_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Подготовка среды

In [ ]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz &> /dev/null
!tar xzf sqlite.tar.gz &> /dev/null
%cd sqlite/
!./configure &> /dev/null
!make sqlite3.c &> /dev/null
%cd /content
!npx degit coleifer/pysqlite3 -f &> /dev/null
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build &> /dev/null
!cp build/lib.linux-x86_64-3.8/pysqlite3/_sqlite3.cpython-38-x86_64-linux-gnu.so \
    /usr/lib/python3.8/lib-dynload/_sqlite3.cpython-38-x86_64-linux-gnu.so

/content/sqlite
/content


In [ ]:
import sqlite3
sqlite3.sqlite_version

'3.40.0'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Задача №1. Процентное изменение месяц к месяцу


## 1.0. Постановка задачи
**Контекст:** часто полезно знать, как изменяется ключевая метрика, например, месячная аудитория активных пользователей, от месяца к месяцу. Допустим у нас есть таблица logins в таком виде:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```
**Задача:** Найти ежемесячное процентное изменение месячной аудитории активных пользователей (MAU).

## 1.1. Подготовка данных

In [ ]:
def random_dates(start, end, n=10):
    '''
    Dividing the unix time values(start and end) by 24*60*60*10**9
    to make it days and return n randomly chosen days back.
    '''
    start_u = start.value//(24*60*60*10**9)
    end_u = end.value//(24*60*60*10**9)

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='D')


In [ ]:
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2022-12-01')
login_lines = 100000
user_ids = 30000

logins = pd.DataFrame()
logins['user_id'] = np.random.randint(1, user_ids, login_lines)
logins['date'] = pd.Series(random_dates(start, end, login_lines))
logins

,user_id,date
0,26233,2022-07-17
1,29742,2022-03-21
2,22565,2022-01-04
3,27239,2021-02-27
4,2625,2021-03-04
...,...,...
99995,22326,2022-01-16
99996,12134,2021-05-29
99997,2937,2021-01-26
99998,26438,2021-10-08


In [ ]:
con = sqlite3.connect('db')
cur = con.cursor()

In [ ]:
logins.to_sql('logins', con, if_exists='replace',index=False)

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
sql = '''select * from logins'''
select(sql)

,user_id,date
0,26233,2022-07-17 00:00:00
1,29742,2022-03-21 00:00:00
2,22565,2022-01-04 00:00:00
3,27239,2021-02-27 00:00:00
4,2625,2021-03-04 00:00:00
...,...,...
99995,22326,2022-01-16 00:00:00
99996,12134,2021-05-29 00:00:00
99997,2937,2021-01-26 00:00:00
99998,26438,2021-10-08 00:00:00


## 1.2. Решение SQL

In [ ]:
sql ='''
with months_active as (
  select 
      strftime('%Y %m', date) as year_month,
      cast(count(user_id) as real) as users
  from
      logins
  group by 
      year_month
)

select 
    year_month,
    users, 
    (users - LAG(users) over(order by year_month))*100/LAG(users) over(order by year_month) MAU_percent
from
    months_active
'''
select(sql)

,year_month,users,MAU_percent
0,2021 01,4484.0,NaN
1,2021 02,4061.0,-9.433541
2,2021 03,4437.0,9.258803
3,2021 04,4312.0,-2.817219
4,2021 05,4341.0,0.672542
5,2021 06,4332.0,-0.207326
6,2021 07,4377.0,1.038781
7,2021 08,4390.0,0.297007
8,2021 09,4364.0,-0.592255
9,2021 10,4332.0,-0.733272


## 1.3 Решение Pandas

In [ ]:
pt = logins.pivot_table(index=[(logins['date'].dt.to_period('M'))], 
                        values=['user_id'],
                        aggfunc="count"
                       )
pt['MAU,%mm'] = (pt['user_id'] - pt['user_id'].shift(1))/pt['user_id'].shift(1)*100
pt

,user_id,"MAU,%mm"
date,,
2021-01,4484,NaN
2021-02,4061,-9.433541
2021-03,4437,9.258803
2021-04,4312,-2.817219
2021-05,4341,0.672542
2021-06,4332,-0.207326
2021-07,4377,1.038781
2021-08,4390,0.297007
2021-09,4364,-0.592255


# 2.Задача №2. Маркировка древовидной структуры

## 2.0 Постановка задачи
Контекст: предположим, у вас есть таблица tree с двумя столбцами: в первом указаны узлы, а во втором — родительские узлы.
```
node   parent
1       2
2       5
3       5
4       3
5       NULL 
```
Задача: написать SQL таким образом, чтобы мы обозначили каждый узел как внутренний (inner), корневой (root) или конечный узел/лист (leaf), так что для вышеперечисленных значений получится следующее:
```
node    label  
1       Leaf
2       Inner
3       Inner
4       Leaf
5       Root
```

## 2.1 Подготовка данных

In [ ]:
nodes = pd.DataFrame({'node': [1,2,3,4,5],
                      'parent':[2,5,5,3,np.nan]})
nodes

,node,parent
0,1,2.0
1,2,5.0
2,3,5.0
3,4,3.0
4,5,NaN


In [ ]:
nodes.to_sql('nodes', con, if_exists='replace',index=False)

### 2.2 Решение SQL

In [ ]:
sql = '''
select
    node,
    case
        when parent is null then 'Root'
        when node in (select parent from nodes) then 'Inner'
        else 'Leaf'
    end as label
from
    nodes
'''
select(sql)

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


## 2.3 Решение Pandas

In [ ]:
nodes['label'] =  np.where(nodes['parent'].isna(), 'Root',
                  np.where(nodes['node'].isin(nodes['parent']), 'Inner', 'Leaf'))
nodes[['node','label']]

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


# 3. Задача № 3. Удержание пользователей в месяц.

## 3.0. Постановка задачи

**Контекст:** допустим, у нас есть статистика по авторизации пользователей на сайте в таблице logins:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```

**Задача:** написать запрос, который получает количество удержанных пользователей в месяц. В нашем случае данный параметр определяется как количество пользователей, которые авторизовались в системе и в этом, и в предыдущем месяце.


## 3.1. Подготовка данных

In [ ]:
logins.to_sql('logins', con, if_exists='replace',index=False)

## 3.2. Решение SQL

In [ ]:
sql = '''
with months_logins as (
  select
      distinct user_id, 
      strftime('%Y %m', date) as cur_month,
      strftime('%Y %m', date(date, 'start of month','-1 month')) as prev_month
  from
      logins
)

select
    m1.cur_month,
    count(m2.user_id)
from
    months_logins as m1
left join 
    months_logins as m2 on
        m1.user_id = m2.user_id 
        and m1.prev_month = m2.cur_month
group by
    m1.cur_month
'''
select(sql)

,cur_month,count(m2.user_id)
0,2021 01,0
1,2021 02,503
2,2021 03,504
3,2021 04,551
4,2021 05,538
5,2021 06,552
6,2021 07,575
7,2021 08,536
8,2021 09,566
9,2021 10,527


## 3.3. Решение Pandas

In [ ]:
logins_extended = logins.copy()
logins_extended['id_and_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + logins['date'].dt.to_period('M').dt.to_timestamp().astype(str))
logins_extended['id_and_prev_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + (logins['date'].dt.to_period('M').dt.to_timestamp() 
       - pd.DateOffset(months=1)).astype(str))
logins_extended = logins_extended.drop_duplicates(['user_id','id_and_month'])
logins_extended

,user_id,date,id_and_month,id_and_prev_month
0,26233,2022-07-17,26233_2022-07-01,26233_2022-06-01
1,29742,2022-03-21,29742_2022-03-01,29742_2022-02-01
2,22565,2022-01-04,22565_2022-01-01,22565_2021-12-01
3,27239,2021-02-27,27239_2021-02-01,27239_2021-01-01
4,2625,2021-03-04,2625_2021-03-01,2625_2021-02-01
...,...,...,...,...
99994,11827,2022-06-10,11827_2022-06-01,11827_2022-05-01
99995,22326,2022-01-16,22326_2022-01-01,22326_2021-12-01
99996,12134,2021-05-29,12134_2021-05-01,12134_2021-04-01
99997,2937,2021-01-26,2937_2021-01-01,2937_2020-12-01


In [ ]:
logins_extended['prev_month_login'] = (
    np.where(logins_extended['id_and_prev_month'].isin(logins_extended['id_and_month'])
             , 1 , 0))
logins_extended.groupby([logins_extended['date'].dt.to_period('M')])['prev_month_login'].sum()

<ipython-input-18-51ef2d549242>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logins_extended['prev_month_login'] = (


date
2021-01      0
2021-02    503
2021-03    504
2021-04    551
2021-05    538
2021-06    552
2021-07    575
2021-08    536
2021-09    566
2021-10    527
2021-11    513
2021-12    595
2022-01    622
2022-02    539
2022-03    475
2022-04    558
2022-05    529
2022-06    556
2022-07    590
2022-08    559
2022-09    553
2022-10    540
2022-11    516
Freq: M, Name: prev_month_login, dtype: int64

# 4. Задача № 4. Нарастающий итог

## 4.0. Поставнока задачи

**Контекст:** допустим, у нас есть таблица transactions в таком виде:

| date       | cash_flow |
|------------|-----------|
| 2018-01-01 | -1000     |
| 2018-01-02 | -100      |
| 2018-01-03 | 50        |
| ...        | ...       |


Где cash_flow — это выручка минус затраты за каждый день.

**Задача:** написать запрос, чтобы получить нарастающий итог для денежного потока каждый день таким образом, чтобы в конечном итоге получилась таблица в такой форме:

| date       | cumulative_cf |
|------------|---------------|
| 2018-01-01 | -1000         |
| 2018-01-02 | -1100         |
| 2018-01-03 | -1050         |
| ...        | ...           |

## 4.1. Подготовка данных

In [ ]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

transactions = pd.DataFrame()
transactions['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
transactions['cash_flow'] = np.random.randint(-2000, 2000, sample_size)
transactions

,date,cash_flow
0,2021-01-01,830
1,2021-01-02,1187
2,2021-01-03,-1875
3,2021-01-04,619
4,2021-01-05,1725
5,2021-01-06,-973
6,2021-01-07,72
7,2021-01-08,-994
8,2021-01-09,-1062
9,2021-01-10,882


In [ ]:
transactions.to_sql('transactions', con, if_exists='replace',index=False)

## 4.2. Решение SQL

In [ ]:
sql = '''
select
    date,
    sum(cash_flow) over (order by date) AS cumulative_cf
from
    transactions
group by
    date
order by
    date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,830
1,2021-01-02 00:00:00,2017
2,2021-01-03 00:00:00,142
3,2021-01-04 00:00:00,761
4,2021-01-05 00:00:00,2486
5,2021-01-06 00:00:00,1513
6,2021-01-07 00:00:00,1585
7,2021-01-08 00:00:00,591
8,2021-01-09 00:00:00,-471
9,2021-01-10 00:00:00,411


## 4.3. Решение SQL без оконной функции

In [ ]:
sql = '''
select
    t1.date,
    sum(t2.cash_flow) as cumulative_cf
from
    transactions as t1
join
    transactions as t2 on t1.date >= t2.date
group by
    t1.date
order by
    t1.date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,830
1,2021-01-02 00:00:00,2017
2,2021-01-03 00:00:00,142
3,2021-01-04 00:00:00,761
4,2021-01-05 00:00:00,2486
5,2021-01-06 00:00:00,1513
6,2021-01-07 00:00:00,1585
7,2021-01-08 00:00:00,591
8,2021-01-09 00:00:00,-471
9,2021-01-10 00:00:00,411


## 4.4. Решение Pandas

In [ ]:
transactions_extended = transactions.copy().sort_values('date')
transactions_extended['cumulative_cf'] = transactions_extended['cash_flow'].cumsum()
transactions_extended[['date','cumulative_cf']]

,date,cumulative_cf
0,2021-01-01,830
1,2021-01-02,2017
2,2021-01-03,142
3,2021-01-04,761
4,2021-01-05,2486
5,2021-01-06,1513
6,2021-01-07,1585
7,2021-01-08,591
8,2021-01-09,-471
9,2021-01-10,411


# 5. Задача № 5. Скользящее среднее

## 5.0. Постановка задачи

**Контекст:** допустим, у нас есть таблица signups в таком виде:

| date       | sign_ups |
|------------|----------|
| 2018-01-01 | 10       |
| 2018-01-02 | 20       |
| 2018-01-03 | 50       |
| ...        | ...      |
| 2018-10-01 | 35       |


**Задача:** написать запрос, чтобы получить 7-дневное скользящее среднее ежедневных регистраций

## 5.1. Подготовка данных

In [ ]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

signups = pd.DataFrame()
signups['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
signups['sign_ups'] = np.random.randint(0, 100, sample_size)
signups

,date,sign_ups
0,2021-01-01,95
1,2021-01-02,62
2,2021-01-03,65
3,2021-01-04,85
4,2021-01-05,71
5,2021-01-06,64
6,2021-01-07,43
7,2021-01-08,42
8,2021-01-09,50
9,2021-01-10,19


In [ ]:
signups.to_sql('signups', con, if_exists='replace',index=False)

## 5.2. Решение SQL

In [ ]:
sql = '''
select
    s1.date,
    s1.sign_ups,
    avg(s2.sign_ups)
from
    signups as s1
join
    signups as s2 on
        s1.date < date(s2.date, '7 days')
        and s1.date >= s2.date
group by
    s1.date
'''

results = select(sql)
results

,date,sign_ups,avg(s2.sign_ups)
0,2021-01-01 00:00:00,95,95.000000
1,2021-01-02 00:00:00,62,78.500000
2,2021-01-03 00:00:00,65,74.000000
3,2021-01-04 00:00:00,85,76.750000
4,2021-01-05 00:00:00,71,75.600000
5,2021-01-06 00:00:00,64,73.666667
6,2021-01-07 00:00:00,43,69.285714
7,2021-01-08 00:00:00,42,61.714286
8,2021-01-09 00:00:00,50,60.000000
9,2021-01-10 00:00:00,19,53.428571


## 5.3. Решение Pandas

In [ ]:
signups_extended = signups.copy().set_index('date')
signups_extended.rolling('7D').mean()

,sign_ups
date,
2021-01-01,95.000000
2021-01-02,78.500000
2021-01-03,74.000000
2021-01-04,76.750000
2021-01-05,75.600000
2021-01-06,73.666667
2021-01-07,69.285714
2021-01-08,61.714286
2021-01-09,60.000000


# 6. Задача № 6. Несколько условий соединения

## 6.0 Постановка задачи

**Контекст:** скажем, наша таблица emails содержит электронные письма, отправленные с адреса zach@g.com и полученные на него:

| id | subject  | from         | to           | timestamp           |
|----|----------|--------------|--------------|---------------------|
| 1  | Yosemite | zach@g.com   | thomas@g.com | 2018-01-02 12:45:03 |
| 2  | Big Sur  | sarah@g.com  | thomas@g.com | 2018-01-02 16:30:01 |
| 3  | Yosemite | thomas@g.com | zach@g.com   | 2018-01-02 16:35:04 |
| 4  | Running  | jill@g.com   | zach@g.com   | 2018-01-03 08:12:45 |
| 5  | Yosemite | zach@g.com   | thomas@g.com | 2018-01-03 14:02:01 |
| 6  | Yosemite | thomas@g.com | zach@g.com   | 2018-01-03 15:01:05 |
| .. | ..       | ..           | ..           | ..                  |


**Задача:** написать запрос, чтобы получить время отклика на каждое письмо (id), отправленное на zach@g.com. Не включать письма на другие адреса. Предположим, что у каждого треда уникальная тема. Имейте в виду, что в треде может быть несколько писем туда и обратно между zach@g.com и другими адресатами.


## 6.1 Подготовка данных

In [ ]:
emails = pd.DataFrame({'subject': ['Yosemite',
                                   'Big Sur',
                                   'Yosemite',
                                   'Running',
                                   'Yosemite',
                                   'Yosemite'],
                       'from' : ['zach@g.com',
                                 'sarah@g.com',
                                 'thomas@g.com',
                                 'jill@g.com',
                                 'zach@g.com',
                                 'thomas@g.com'],
                       'to' : ['thomas@g.com',
                               'thomas@g.com',
                               'zach@g.com',
                               'zach@g.com',
                               'thomas@g.com',
                               'zach@g.com'],
                       'date': [pd.Timestamp('2018-01-02 12:45:03'),
                                pd.Timestamp('2018-01-02 16:30:01'),
                                pd.Timestamp('2018-01-02 16:35:04'),
                                pd.Timestamp('2018-01-03 08:12:45'),
                                pd.Timestamp('2018-01-03 14:02:01'),
                                pd.Timestamp('2018-01-03 15:01:05')]
                       })
emails

,subject,from,to,date
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03
1,Big Sur,sarah@g.com,thomas@g.com,2018-01-02 16:30:01
2,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04
3,Running,jill@g.com,zach@g.com,2018-01-03 08:12:45
4,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01
5,Yosemite,thomas@g.com,zach@g.com,2018-01-03 15:01:05


In [ ]:
emails.to_sql('emails', con, if_exists='replace')

## 6.2. Решение SQL

In [ ]:
sql = '''
select
    e1.subject,
    e1."from",
    e1."to",
    e1.date,
    e2.date,
    CAST((JulianDay(min(e2.date)) - JulianDay(e1.date)) * 24 as Integer) as diff_in_hours
from
    emails as e1
join
    emails as e2 on
        e1.subject = e2.subject
        and e1."from" = e2."to"
        and e1."to" = e2."from"
        and e1.date < e2.date
where e1."to" == "zach@g.com"
group by e1."index"
'''

select(sql)

,subject,from,to,date,date,diff_in_hours
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03,2018-01-02 16:35:04,3
1,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04,2018-01-03 14:02:01,21
2,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01,2018-01-03 15:01:05,0


## 6.3. Решение Pandas

In [ ]:
emails_extended = pd.merge(emails,
                           emails,
                           how='left',
                           left_on=['subject','from','to'],
                           right_on = ['subject','to','from']
                          ).query('date_x < date_y')
emails_extended['diff_in_hours'] = (emails_extended['date_y'] -
                                    emails_extended['date_x']
                                   )
emails_extended.groupby(['subject',
                         'from_x',
                         'to_x',
                         'date_x'
                        ]).agg({'date_y':'min',
                                'diff_in_hours':'min'
                               }).sort_values(['date_x']).reset_index()

,subject,from_x,to_x,date_x,date_y,diff_in_hours
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03,2018-01-02 16:35:04,0 days 03:50:01
1,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04,2018-01-03 14:02:01,0 days 21:26:57
2,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01,2018-01-03 15:01:05,0 days 00:59:04


# 7. Задача № 7. Найти идентификатор с максимальным значением

## 7.0. Постановка задачи

**Контекст:** Допустим, у нас есть таблица salaries с данными об отделах и зарплате сотрудников в следующем формате:

|  depname  | empno | salary |     
|-----------|-------|--------|
| develop   |    11 |   5200 | 
| develop   |     7 |   4200 | 
| develop   |     9 |   4500 | 
| develop   |     8 |   6000 | 
| develop   |    10 |   5200 | 
| personnel |     5 |   3500 | 
| personnel |     2 |   3900 | 
| sales     |     3 |   4800 | 
| sales     |     1 |   5000 | 
| sales     |     4 |   4800 |

**Задача:** написать запрос, чтобы получить empno с самой высокой зарплатой. Убедитесь, что ваше решение обрабатывает случаи одинаковых зарплатами!


## 7.1. Подготовка данных

In [ ]:
sample_size = 10
salaries = pd.DataFrame()
salaries['depname'] = np.random.choice(['develop','personnel','sales'],
                                       sample_size)
salaries['empno'] = np.random.choice(range(sample_size),
                                       sample_size,
                                       replace=False)
salaries['salary'] = np.random.randint(4000, 12000, sample_size)
salaries

,depname,empno,salary
0,sales,1,6765
1,sales,8,6602
2,sales,5,6925
3,sales,2,5925
4,personnel,9,5787
5,develop,3,5224
6,personnel,7,4784
7,sales,4,9603
8,sales,0,6759
9,develop,6,7421


In [ ]:
salaries.to_sql('salaries', con, if_exists='replace', index=False)

## 7.2. Решение SQL

In [ ]:
sql = '''
with max_salary as (
  select
      max(salary) as max_salary
  from
      salaries
)

select
    empno
from
    salaries
join max_salary on
    salaries.salary = max_salary.max_salary
'''
select(sql)

,empno
0,4


## 7.3. Решение Pandas

In [ ]:
salaries[salaries['salary'] == salaries['salary'].max()]['empno']

7    4
Name: empno, dtype: int64

# 8. Задача № 8. Среднее значение

## 8.0. Постановка задачи

**Задача:** спользую наббор данных из предыдушей задачи написать запрос, который возвращает ту же таблицу, но с новым столбцом, в котором указана средняя зарплата по департаменту. Мы бы ожидали таблицу в таком виде:

  depname  | empno | salary | avg_salary |     
-----------|-------|--------|------------|
 develop   |    11 |   5200 |       5020 |
 develop   |     7 |   4200 |       5020 | 
 develop   |     9 |   4500 |       5020 |
 develop   |     8 |   6000 |       5020 | 
 develop   |    10 |   5200 |       5020 | 
 personnel |     5 |   3500 |       3700 |
 personnel |     2 |   3900 |       3700 |
 sales     |     3 |   4800 |       4867 | 
 sales     |     1 |   5000 |       4867 | 
 sales     |     4 |   4800 |       4867 |

## 8.1. Решение SQL

In [ ]:
sql = '''
select
    *,
    avg(salary) OVER (PARTITION BY depname) as avg_salary
from
    salaries
'''
select(sql)

,depname,empno,salary,avg_salary
0,develop,3,5224,6322.5
1,develop,6,7421,6322.5
2,personnel,9,5787,5285.5
3,personnel,7,4784,5285.5
4,sales,1,6765,7096.5
5,sales,8,6602,7096.5
6,sales,5,6925,7096.5
7,sales,2,5925,7096.5
8,sales,4,9603,7096.5
9,sales,0,6759,7096.5


## 8.2. Решение Pandas

In [ ]:
salaries_extended = salaries.copy()
salaries_extended['avg_price'] = salaries_extended.groupby(['depname'])['salary'].transform('mean')
salaries_extended.sort_values('depname')

,depname,empno,salary,avg_price
5,develop,3,5224,6322.5
9,develop,6,7421,6322.5
4,personnel,9,5787,5285.5
6,personnel,7,4784,5285.5
0,sales,1,6765,7096.5
1,sales,8,6602,7096.5
2,sales,5,6925,7096.5
3,sales,2,5925,7096.5
7,sales,4,9603,7096.5
8,sales,0,6759,7096.5


# 9. Задача № 9. Ранжирование с оконной функцией

## 9.0. Постановка задачи

**Задача:** напишите запрос, который добавляет столбец с позицией каждого сотрудника в табели на основе его зарплаты в своём отделе, где сотрудник с самой высокой зарплатой получает позицию 1. Мы бы ожидали таблицу в таком виде:

  depname  | empno | salary | salary_rank |     
-----------|-------|--------|-------------|
 develop   |    11 |   5200 |           2 |
 develop   |     7 |   4200 |           5 | 
 develop   |     9 |   4500 |           4 |
 develop   |     8 |   6000 |           1 | 
 develop   |    10 |   5200 |           2 | 
 personnel |     5 |   3500 |           2 |
 personnel |     2 |   3900 |           1 |
 sales     |     3 |   4800 |           2 | 
 sales     |     1 |   5000 |           1 | 
 sales     |     4 |   4800 |           2 | 

## 9.1. Решение SQL

In [ ]:
sql = '''
select
    *,
    rank() over (partition by depname order by salary desc)
from
    salaries
'''
select(sql)

,depname,empno,salary,rank() over (partition by depname order by salary desc)
0,develop,6,7421,1
1,develop,3,5224,2
2,personnel,9,5787,1
3,personnel,7,4784,2
4,sales,4,9603,1
5,sales,5,6925,2
6,sales,1,6765,3
7,sales,0,6759,4
8,sales,8,6602,5
9,sales,2,5925,6


## 9.2. Решение Pandas

In [ ]:
salaries_extended = salaries.copy()
salaries_extended['rank'] = salaries_extended.groupby("depname")['salary'].rank(ascending=False)
salaries_extended.sort_values(['depname','rank'])

,depname,empno,salary,rank
9,develop,6,7421,1.0
5,develop,3,5224,2.0
4,personnel,9,5787,1.0
6,personnel,7,4784,2.0
7,sales,4,9603,1.0
2,sales,5,6925,2.0
0,sales,1,6765,3.0
8,sales,0,6759,4.0
1,sales,8,6602,5.0
3,sales,2,5925,6.0


# 10. Задача № 10. Гистограммы

## 10.0 Постановка задачи

**Контекст:** Допустим, у нас есть таблица sessions, где каждая строка представляет собой сеанс потоковой передачи видео с длиной в секундах:

| session_id | length_seconds |
|------------|----------------|
| 1          | 23             |
| 2          | 453            |
| 3          | 27             |
| ..         | ..             |


**Задача:** написать запрос, чтобы подсчитать количество сеансов, которые попадают промежутки по пять секунд, т. е. для приведённого выше фрагмента результат будет примерно такой:

| bucket  | count |
|---------|-------|
| 20-25   | 2     |
| 450-455 | 1     |

## 10.1. Подготовка данных

In [ ]:
sample_size = 10
sessions = pd.DataFrame()
sessions['session_id'] = range(sample_size)
sessions['length_seconds'] = np.random.randint(0, 21, sample_size)
sessions

,session_id,length_seconds
0,0,20
1,1,15
2,2,5
3,3,2
4,4,14
5,5,9
6,6,15
7,7,0
8,8,4
9,9,0


In [ ]:
sessions.to_sql('sessions', con, if_exists='replace', index=False)

## 10.2. Решение SQL

In [ ]:
sql = '''
with buckets as (
  select session_id,
         length_seconds/5 as bucket_num
  from sessions
)
select
  '[' || cast(bucket_num*5 as text) || ',' || cast(bucket_num*5+5 as text) || + ')' as bucket,
  count(session_id) as count
from
    buckets
group by
    bucket_num
order by
    bucket_num
'''
select(sql)

,bucket,count
0,"[0,5)",4
1,"[5,10)",2
2,"[10,15)",1
3,"[15,20)",2
4,"[20,25)",1


## 10.3. Решение Pandas

In [ ]:
list(range(0,sessions['length_seconds'].max() + 6,5))

[0, 5, 10, 15, 20, 25]

In [ ]:
sessions_extended = sessions.copy()
sessions_extended['bucket'] = pd.cut(sessions_extended['length_seconds'],
                                     range(0, sessions_extended['length_seconds'].max()+6,5),
                                     right=False)
sessions_extended.groupby('bucket')['session_id'].count()

bucket
[0, 5)      4
[5, 10)     2
[10, 15)    1
[15, 20)    2
[20, 25)    1
Name: session_id, dtype: int64

# 11. Задача № 11. Перекрёстное соединение

## 11.0. Постанвока задачи

**Контекст: ** допустим, у нас есть таблица state_streams, где в каждой строке указано название штата и общее количество часов потоковой передачи с видеохостинга:

| state | total_streams |
|-------|---------------|
| NC    | 34569         |
| SC    | 33999         |
| CA    | 98324         |
| MA    | 19345         |
| ..    | ..            |

**Задача:** написать запрос, чтобы получить пары штатов с общим количеством потоков в пределах тысячи друг от друга. Для приведённого выше фрагмента мы хотели бы увидеть что-то вроде:

| state_a | state_b |
|---------|---------|
| NC      | SC      |
| SC      | NC      |

## 11.1. Подготовка данных

In [ ]:
state_streams = pd.DataFrame({'state':['NC',
                                       'SC',
                                       'CA',
                                       'MA',
                                       ],
                              'total_streams':[34569,
                                               33999,
                                               98324,
                                               19345
                                               ]
                              })
state_streams

,state,total_streams
0,NC,34569
1,SC,33999
2,CA,98324
3,MA,19345


In [ ]:
state_streams.to_sql('state_streams', con, if_exists='replace', index=False)

## 11.1. Решение SQL

In [ ]:
sql = '''
select
    s1.state,
    s2.state
from
    state_streams as s1
join
    state_streams as s2 on
        s1.total_streams < s2.total_streams + 1000
        and s1.total_streams > s2.total_streams - 1000
        and s1.state != s2.state
'''
select(sql)

,state,state
0,NC,SC
1,SC,NC


## 11.2. Решение Pandas

**Комментарий:** Конкретно для этой задачи лучше использовать SQL. В Pandas мы сначала должны объеденить таблицы и только потом можем применить фильтр.

In [ ]:
results = state_streams.merge(state_streams, how='cross')
results.query("total_streams_x < total_streams_y + 1000 and \
               total_streams_x > total_streams_y - 1000 and \
               state_x != state_y")

,state_x,total_streams_x,state_y,total_streams_y
1,NC,34569,SC,33999
4,SC,33999,NC,34569
